In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Start from here:**

In [1]:
############# Starts from here ##################
import json
import numpy as np

ModuleNotFoundError: No module named 'numpy'

In [ ]:
with open('captures_000.json') as f:
  data = json.load(f)

#first remove version
del data['version']

In [ ]:
#Second remove the unrequired info from captures
print(len(data['captures']))

x_translation_list_rock = []
y_translation_list_rock = []
z_translation_list_rock = []
x_rotation_list_rock = []
y_rotation_list_rock = []
z_rotation_list_rock = []
w_rotation_list_rock = []

x_translation_list_cam = []
y_translation_list_cam = []
z_translation_list_cam = []
x_rotation_list_cam = []
y_rotation_list_cam = []
z_rotation_list_cam = []
w_rotation_list_cam = []

for capture in data['captures']:
  translation = capture['sensor']['translation']
  rotation = capture['sensor']['rotation']
  # # print(translation)
  x_translation_list_cam.append(translation[0])
  y_translation_list_cam.append(translation[1])
  z_translation_list_cam.append(translation[2])
  x_rotation_list_cam.append(rotation[0])
  y_rotation_list_cam.append(rotation[1])
  z_rotation_list_cam.append(rotation[2])
  w_rotation_list_cam.append(rotation[3])
  # print(rotation)
  ##Extract the rock poses
  for annotation in capture['annotations']:
      for value in annotation['values']:
        translation = value['translation']
        rotation = value['rotation']
        # print(translation)
        x_translation_list_rock.append(translation['x'])
        y_translation_list_rock.append(translation['y'])
        z_translation_list_rock.append(translation['z'])
        x_rotation_list_rock.append(rotation['x'])
        y_rotation_list_rock.append(rotation['y'])
        z_rotation_list_rock.append(rotation['z'])
        w_rotation_list_rock.append(rotation['w'])

    
  # #inside each annotation (rock_position), keep only the translations and quaternions
  # for item in capture['annotations']:
  #   del item['id']
  #   del item['annotation_definition']
    
  #   ##inside the annotations, access the values to get only the translation and rotation
  #   for value in a7a['values']:
  #     del value['label_id']
  #     del value['instance_id']
  #     del value['size']
  #     del value['velocity']
  #     del value['acceleration']


with open('rock_captures.json', 'w') as f:
  json.dump(data, f, indent=2)

1295


In [ ]:
# x_rotation_list_cam

In [ ]:
# x_translation_list_rock = np.array(x_translation_list_rock)
# y_translation_list_rock = np.array(y_translation_list_rock)
# z_translation_list_rock = np.array(z_translation_list_rock)
# x_rotation_list_rock = np.array(x_rotation_list_rock)
# y_rotation_list_rock = np.array(y_rotation_list_rock)
# z_rotation_list_rock = np.array(z_rotation_list_rock)
# w_rotation_list_rock = np.array(w_rotation_list_rock)

#Converting to numpy array:

x_translation_list_cam = np.array(x_translation_list_cam)
y_translation_list_cam = np.array(y_translation_list_cam)
z_translation_list_cam = np.array(z_translation_list_cam)
x_rotation_list_cam = np.array(x_rotation_list_cam)
y_rotation_list_cam = np.array(y_rotation_list_cam)
z_rotation_list_cam = np.array(z_rotation_list_cam)
w_rotation_list_cam = np.array(w_rotation_list_cam)

In [ ]:
#substracting camera poses from rock poses
#Also check the coordinates

# x_translation = -x_translation_list_rock + x_translation_list_cam
# y_translation = -y_translation_list_rock + y_translation_list_cam
# z_translation = -z_translation_list_rock + z_translation_list_cam

# q1 = -x_rotation_list_rock + x_rotation_list_cam
# q2 = -y_rotation_list_rock + y_rotation_list_cam
# q3 = -z_rotation_list_rock + z_rotation_list_cam
# q4 = -w_rotation_list_rock + w_rotation_list_cam

In [ ]:
## Only the camera pose ##
x_translation = x_translation_list_cam
y_translation = y_translation_list_cam
z_translation = z_translation_list_cam

q1 = x_rotation_list_cam
q2 = y_rotation_list_cam
q3 = z_rotation_list_cam
q4 = w_rotation_list_cam

# pose = np.stack([ z_translation, x_translation, y_translation, q1, q2, q3, q4], axis=1)

In [ ]:
# import numpy as np

timestamp = np.arange(0.0,float(len(x_translation)), )

In [ ]:
len(timestamp)
len(x_translation)
# timestamp

1295

Pose File for TartanVO:

The TUM (Technical University of Munich) format is a standard for representing 3D poses (position and orientation) in a robotics or computer vision application. It specifies a coordinate frame with the x-axis pointing to the right, the y-axis pointing up, and the z-axis pointing forward.

To convert from a coordinate frame with the y-axis pointing up and the z-axis pointing forward to the TUM format, you can use a coordinate transformation matrix. This matrix will rotate the y-axis to the x-axis, the z-axis to the y-axis, and the x-axis (which was not mentioned in the original coordinate frame) to the z-axis.

Here's an example of how you can define this coordinate transformation matrix in Python:

In [ ]:
# arr = np.array([x_rotation_list_cam, y_rotation_list_cam, z_rotation_list_cam])
# print(arr.shape)
# for line in arr:
#   print(line[:3])

(3, 1295)
[-4.37113954e-08 -4.37113954e-08 -4.37113954e-08]
[-1.00000012 -1.00000012 -1.00000012]
[0. 0. 0.]


In [ ]:
# import numpy as np

# # Define the coordinate transformation matrix
T = np.array([[0, 0, 1],  # Rotate z-axis to y-axis
              [1, 0, 0],  # Rotate y-axis to x-axis
              [0, 1, 0]]) # Rotate x-axis to y-axis

# Apply the coordinate transformation to a point or pose
point = np.array([x_rotation_list_cam, y_rotation_list_cam, z_rotation_list_cam])
point_tum = np.dot(T, point)

In [ ]:
rotation = np.array([[1 - 2*q2**2 - 2*q3**2, 2*q1*q2 - 2*q3*q4, 2*q1*q3 + 2*q2*q4],
                     [2*q1*q2 + 2*q3*q4, 1 - 2*q1**2 - 2*q3**2, 2*q2*q3 - 2*q1*q4],
                     [2*q1*q3 - 2*q2*q4, 2*q2*q3 + 2*q1*q4, 1 - 2*q1**2 - 2*q2**2]])

# Apply the coordinate transformation to the rotation part of the pose
rotation_tum = np.dot(T, rotation)

# Concatenate the rotated rotation and the original translation to form the transformed pose
pose_tum = np.concatenate((rotation_tum, [x_translation, y_translation, z_translation]), axis=1)

In [ ]:
print(point_tum[1].shape)

(1295,)


In [ ]:
pose = np.stack([point_tum[0], point_tum[1], point_tum[2], q1, q2, q3, q4], axis=1)

[Conversion Left Hand Coordinates to Right](https://https://studylib.net/doc/18222978/conversion-of-left-handed-coordinates-to-right)


In [ ]:
# Note: Going from Unity world space to ROS world space requires a conversion. Unity's coordinate space has x Right,
# y Up, and z Forward (hence "RUF" coordinates); ROS has x Forward, y Left and z Up (hence "FLU").
# So a Unity (x,y,z) coordinate is equivalent to the ROS (z,-x,y) coordinate. 
# These conversions are done by the To<FLU> function in the ROS-TCP-Connector package's ROSGeometry component.
# pose = np.stack([x_translation, z_translation, y_translation, q1, q2, q3, q4], axis=1)

pose = np.stack([z_translation, -x_translation, y_translation, -q1, -q2, -q3, -q4], axis=1)

According to the output in the ego motion rotation [0 0 0 1], seems the convention is vector-scalar, while according to EuRoC paper, the convention in Hamilton form which is scalar-vector.

In [ ]:
pose = np.stack([z_translation, -x_translation, y_translation, -q4, -q1, -q2, -q3], axis=1)

In [ ]:
#Second option - I checked with ROSGeometry conversion TO<FLU> in Unity
pose = np.stack([z_translation, -x_translation, y_translation, q3, -q1, q2, q4], axis=1)

In [ ]:
np.savetxt('pose_left.txt', pose, delimiter=' ')

swapped the coordinates

In [ ]:
# pose = np.stack([x_translation, y_translation, z_translation, q1, q2, q3, q4], axis=1)
# pose = np.stack([x_translation, z_translation, y_translation, q1, q2, q3, q4], axis=1)
pose = np.stack([timestamp, z_translation, -x_translation, y_translation, q1, q2, q3, q4], axis=1)

# pose

In [ ]:
# from numpy import savetxt
np.savetxt('pose_evo.txt', pose, delimiter=' ')

In [ ]:
#without the camera

# x_translation = -x_translation_list_rock# + x_translation_list_cam
# y_translation = -y_translation_list_rock #+ y_translation_list_cam
# z_translation = -z_translation_list_rock #+ z_translation_list_cam

# q1 = -x_rotation_list_rock# + x_rotation_list_cam
# q2 = -y_rotation_list_rock# + y_rotation_list_cam
# q3 = -z_rotation_list_rock# + z_rotation_list_cam
# q4 = -w_rotation_list_rock# + w_rotation_list_cam

In [ ]:
tartan_result = np.loadtxt('unity_vo_model_pretrained.txt', delimiter=' ')

In [ ]:
print('timestamp shape', timestamp.shape)
timestamp2 = np.expand_dims(timestamp, axis=1)
print('timestamp shape after expanding the dimensions', timestamp2.shape)
print('pose shape', tartan_result.shape)

timestamp shape (10001,)
timestamp shape after expanding the dimensions (10001, 1)
pose shape (10001, 7)


In [ ]:
tartan_result = np.concatenate((timestamp2, tartan_result), axis=1)

In [ ]:
np.savetxt('tartan_evo.txt', tartan_result, delimiter=' ')

In [ ]:
# tartan_result = pd.read_csv('unity_tartanvo_1914.csv', delimiter=' ')

In [ ]:
# a7a.insert(0, column = 0, value = timestamp)

In [ ]:
# a7a.head()